In [1]:
import neuralnet as nn
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
from typing import List, Any
np.random.seed(123)

In [ ]:
class Pooler(nn.Model2D):
    def __init__(self):
        super().__init__()
        self._layer1 = nn.convolution.MaxPool2D()
        self._layer2 = nn.convolution.MaxPool2D()
    
    def forward(self, input):
        h = self._layer1.forward(input)
        print(np.round(h[:,:,0]))
        return  self._layer2.forward(h)
    
    def backward(self, error):
        return  self._layer1.backward(
                    self._layer2.backward(error)
                    )
    
    def update(self, rate):
        self._layer1.update(rate)
        self._layer2.update(rate)


class UnPooler(nn.Model2D):
    def __init__(self, pooler):
        super().__init__()
        self._layer1 = nn.convolution.MaxUnPool2D(pooler._layer2)
        self._layer2 = nn.convolution.MaxUnPool2D(pooler._layer1)
    
    def forward(self, input):
        h = self._layer1.forward(input)
        print(np.round(h[:,:,0]))
        return  self._layer2.forward(h)
    
    def backward(self, error):
        return  self._layer1.backward(
                    self._layer2.backward(error)
                    )
    
    def update(self, rate):
        self._layer1.update(rate)
        self._layer2.update(rate)

In [3]:
pool = Pooler()
unpool = UnPooler(pool)

In [4]:
data = np.array([
    [1, 2, 1, 0,-1, 0, 7, 0],
    [1, 1,-1, 1,-1, 5,-3, 9],
    [0, 0, 3, 2, 1, 1, 8, 0],
    [0, 0, 1, 0, 1,-1, 0, 0],
    [1, 2, 1, 0, 7, 7, 4,-1],
    [1, 3, 1, 0, 1, 9,-9, 0],
    [1, 2, 1, 0, 7, 7, 4,-1],
    [1, 1,-1, 1,-1, 5,-3, 9],
]).reshape((8,8,1))
pooled = np.array([
    [np.mean(data[:4,:4,0]), np.mean(data[:4,4:,0])],
    [np.mean(data[:4,:4,0]), np.mean(data[4:,4:,0])]
]).reshape((2,2,1))
unpooled = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 9],
    [0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 3, 0, 0, 0, 9, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
]).reshape((8,8,1))

In [5]:
np.round(data[:,:,0])

array([[ 1,  2,  1,  0, -1,  0,  7,  0],
       [ 1,  1, -1,  1, -1,  5, -3,  9],
       [ 0,  0,  3,  2,  1,  1,  8,  0],
       [ 0,  0,  1,  0,  1, -1,  0,  0],
       [ 1,  2,  1,  0,  7,  7,  4, -1],
       [ 1,  3,  1,  0,  1,  9, -9,  0],
       [ 1,  2,  1,  0,  7,  7,  4, -1],
       [ 1,  1, -1,  1, -1,  5, -3,  9]])

In [6]:
np.round(pooled[:,:,0])

array([[1., 2.],
       [1., 3.]])

In [7]:
np.round(unpooled[:,:,0])

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 9],
       [0, 0, 3, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 3, 0, 0, 0, 9, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [8]:
out = pool.forward(np.concatenate((data,data),axis=2))
np.round(out[:,:,0])

[[2. 1. 5. 7.]
 [0. 3. 1. 8.]
 [3. 1. 9. 4.]
 [2. 1. 7. 4.]]


array([[3., 5.],
       [3., 9.]])

In [9]:
out2 = unpool.forward(out)
np.round(out2[:,:,0])

[[0. 0. 5. 0.]
 [0. 3. 0. 0.]
 [3. 0. 9. 0.]
 [0. 0. 0. 0.]]
0,0 --> 1,1
0,0 --> 1,1
1,0 --> 2,0
1,0 --> 2,0
0,1 --> 0,2
0,1 --> 0,2
1,1 --> 2,2
1,1 --> 2,2
0,0 --> 1,0
0,0 --> 1,0
1,0 --> 3,1
1,0 --> 3,1
2,0 --> 5,1
2,0 --> 5,1
3,0 --> 6,0
3,0 --> 6,0
0,1 --> 0,3
0,1 --> 0,3
1,1 --> 2,2
1,1 --> 2,2
2,1 --> 4,3
2,1 --> 4,3
3,1 --> 6,2
3,1 --> 6,2
0,2 --> 1,5
0,2 --> 1,5
1,2 --> 2,5
1,2 --> 2,5
2,2 --> 5,5
2,2 --> 5,5
3,2 --> 6,4
3,2 --> 6,4
0,3 --> 1,6
0,3 --> 1,6
1,3 --> 2,6
1,3 --> 2,6
2,3 --> 4,6
2,3 --> 4,6
3,3 --> 6,6
3,3 --> 6,6


array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 5., 0., 0.],
       [0., 0., 3., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0., 9., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
b1 = unpool.backward(out2)
np.round(b1[:,:,0])

array([[3., 5.],
       [3., 9.]])

In [11]:
b2 = pool.backward(b1)
np.round(b2[:,:,0])

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 5., 0., 0.],
       [0., 0., 3., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0., 9., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])